In [20]:
import uproot
import numpy as np
import ROOT

uproot.default_library = "np"

In [21]:
file = uproot.open("nanoaod_kk2f4146_qqpy_91.25_40262.sdst_ttree.root")
tree = file["t"]

In [78]:
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Btag_probAllIP       | RVec<float>              | AsJagged(AsDtype('>f4'), he...
Btag_probNegIP       | RVec<float>              | AsJagged(AsDtype('>f4'), he...
Btag_probPosIP       | RVec<float>              | AsJagged(AsDtype('>f4'), he...
Btag_thrustVector    | ROOT::Math::Displacem... | AsGroup(<TBranchElement 'Bt...
Btag_thrustVector... | ROOT::Math::Cartesian... | AsGroup(<TBranchElement 'fC...
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | ROOT::Math::Cartesian... | AsGroup(<TBranchElement 'Bt...
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | floa

In [73]:
GenAlias           = {}
GenAlias["px"]     = "GenPart_vector/GenPart_vector.fCoordinates.fX"
GenAlias["py"]     = "GenPart_vector/GenPart_vector.fCoordinates.fY"
GenAlias["pz"]     = "GenPart_vector/GenPart_vector.fCoordinates.fZ"
GenAlias["E"]      = "GenPart_vector/GenPart_vector.fCoordinates.fT"
GenAlias["mass"]   = "GenPart_mass"
GenAlias["id"]     = "GenPart_pdgId"
GenAlias["status"] = "GenPart_status"

AllSelection = "(status == 1)"
PionSelection = "(status == 1) & ((id == 211) | (id == -211))"
KaonSelection = "(status == 1) & ((id == 321) | (id == -321))"

Gen = tree.arrays(["px", "py", "pz", "E", "mass", "id", "status"], AllSelection, aliases = GenAlias)

RecoAlias           = {}
RecoAlias["px"]     = "Part_fourMomentum/Part_fourMomentum.fCoordinates.fX"
RecoAlias["py"]     = "Part_fourMomentum/Part_fourMomentum.fCoordinates.fY"
RecoAlias["pz"]     = "Part_fourMomentum/Part_fourMomentum.fCoordinates.fZ"
RecoAlias["E"]      = "Part_fourMomentum/Part_fourMomentum.fCoordinates.fT"
RecoAlias["charge"] = "Part_charge"
RecoAlias["id"]     = "Part_pdgId"

Reco = tree.arrays(["px", "py", "pz", "E", "charge", "id"], aliases = RecoAlias)

PIDAlias = {}
PIDAlias["dedx"]        = "Dedx_valueVD"
PIDAlias["ElectronTag"] = 'Haidc_electronTag'
PIDAlias["HeavyTag"]    = 'Haidc_heavyTag'
PIDAlias["KaonTag"]     = 'Haidc_kaonTag'
PIDAlias["PionTag"]     = 'Haidc_pionTag'
PIDAlias["ProtonTag"]   = 'Haidc_protonTag'
PIDAlias["Selection"]   = 'Haidc_selectionFlag'
PIDAlias["QKaonTag"]    = "Haid_kaonCombined"
PIDAlias["QProtonTag"]  = "Haid_protonCombined"

PID = tree.arrays(["dedx", "ElectronTag", "HeavyTag", "PionTag", "KaonTag", "ProtonTag", "Selection", "QKaonTag", "QProtonTag"], aliases = PIDAlias)

EventAlias = {}
EventAlias["Ecm"]   = "Event_cmEnergy"
EventAlias["Nch"]   = 'Event_chargedMult'
EventAlias["Run"]   = "Event_runNumber"
EventAlias["Event"] = "Event_evtNumber"
EventAlias["Fill"]  = "Event_fillNumber"

Event = tree.arrays(["Ecm", "Nch", "Run", "Event", "Fill"], aliases = EventAlias)

In [67]:
def Magnitude(px, py, pz):
    return np.sqrt(px * px + py * py + pz * pz)

def DotProduct(px1, py1, pz1, px2, py2, pz2):
    return px1 * px2 + py1 * py2 + pz1 * pz2

def CosAngle(px1, py1, pz1, px2, py2, pz2):
    return DotProduct(px1, py1, pz1, px2, py2, pz2) / Magnitude(px1, py1, pz1) / Magnitude(px2, py2, pz2)

def GetAngle(px1, py1, pz1, px2, py2, pz2):
    Value = CosAngle(px1, py1, pz1, px2, py2, pz2)
    if Value < -1:
        Value = -1
    if Value > 1:
        Value = 1
    return np.arccos(Value)

In [77]:
MAX = 10000

outputfile = ROOT.TFile("reduced_nanoaod_kk2f4146_qqpy_91.25_40262.sdst_ttree.root", "RECREATE")
outputtree = ROOT.TTree("Tree", "Strangeness enhancement tree v0")

EventEcm   = np.array([0.0])
EventNch   = np.array([0])
EventRun   = np.array([0])
EventEvent = np.array([0])
EventFill  = np.array([0])

outputtree.Branch("Ecm",   EventEcm,   "Ecm/D")
outputtree.Branch("Nch",   EventNch,   "Nch/L")
outputtree.Branch("Run",   EventRun,   "Run/L")
outputtree.Branch("Event", EventEvent, "Event/L")
outputtree.Branch("Fill",  EventFill,  "Fill/L")

NGen          = np.array([0])
GenPx         = np.array(MAX * [0.0])
GenPy         = np.array(MAX * [0.0])
GenPz         = np.array(MAX * [0.0])
GenE          = np.array(MAX * [0.0])
GenM          = np.array(MAX * [0.0])
GenID         = np.array(MAX * [0])
GenStatus     = np.array(MAX * [0])
GenMatchIndex = np.array(MAX * [0])
GenMatchAngle = np.array(MAX * [0.0])

outputtree.Branch("NGen",          NGen,          "NGen/L")
outputtree.Branch("GenPx",         GenPx,         "GenPx[NGen]/D")
outputtree.Branch("GenPy",         GenPy,         "GenPy[NGen]/D")
outputtree.Branch("GenPz",         GenPz,         "GenPz[NGen]/D")
outputtree.Branch("GenE",          GenE,          "GenE[NGen]/D")
outputtree.Branch("GenM",          GenM,          "GenM[NGen]/D")
outputtree.Branch("GenID",         GenID,         "GenID[NGen]/L")
outputtree.Branch("GenStatus",     GenStatus,     "GenStatus[NGen]/L")
outputtree.Branch("GenMatchIndex", GenMatchIndex, "GenMatchIndex[NGen]/L")
outputtree.Branch("GenMatchAngle", GenMatchAngle, "GenMatchAngle[NGen]/D")

NReco           = np.array([0])
RecoPx          = np.array(MAX * [0.0])
RecoPy          = np.array(MAX * [0.0])
RecoPz          = np.array(MAX * [0.0])
RecoE           = np.array(MAX * [0.0])
RecoCharge      = np.array(MAX * [0.0])
RecoID          = np.array(MAX * [0])
RecoPIDElectron = np.array(MAX * [0])
RecoPIDProton   = np.array(MAX * [0])
RecoPIDPion     = np.array(MAX * [0])
RecoPIDKaon     = np.array(MAX * [0])
RecoPIDHeavy    = np.array(MAX * [0])
RecoPIDQProton  = np.array(MAX * [0.0])
RecoPIDQKaon    = np.array(MAX * [0.0])

outputtree.Branch("NReco",           NReco,           "NReco/L")
outputtree.Branch("RecoPx",          RecoPx,          "RecoPx[NReco]/D")
outputtree.Branch("RecoPy",          RecoPy,          "RecoPy[NReco]/D")
outputtree.Branch("RecoPz",          RecoPz,          "RecoPz[NReco]/D")
outputtree.Branch("RecoE",           RecoE,           "RecoE[NReco]/D")
outputtree.Branch("RecoCharge",      RecoCharge,      "RecoCharge[NReco]/D")
outputtree.Branch("RecoID",          RecoID,          "RecoID[NReco]/L")
outputtree.Branch("RecoPIDElectron", RecoPIDElectron, "RecoPIDElectron[NReco]/L")
outputtree.Branch("RecoPIDProton",   RecoPIDProton,   "RecoPIDProton[NReco]/L")
outputtree.Branch("RecoPIDKaon",     RecoPIDKaon,     "RecoPIDKaon[NReco]/L")
outputtree.Branch("RecoPIDPion",     RecoPIDPion,     "RecoPIDPion[NReco]/L")
outputtree.Branch("RecoPIDHeavy",    RecoPIDHeavy,    "RecoPIDHeavy[NReco]/L")
outputtree.Branch("RecoPIDQProton",  RecoPIDQProton,  "RecoPIDQProton[NReco]/D")
outputtree.Branch("RecoPIDQKaon",    RecoPIDQKaon,    "RecoPIDQKaon[NReco]/D")

# now we loop over the tree and fill things in
EntryCount = len(Gen)
for iE in range(EntryCount):

    # Event level quantities
    EventEcm[0]   = Event[iE].Ecm
    EventNch[0]   = Event[iE].Nch
    EventRun[0]   = Event[iE].Run
    EventEvent[0] = Event[iE].Event
    EventFill[0]  = Event[iE].Fill
    
    # Gen particles
    NGen[0] = len(Gen[iE].px)
    GenPx[0:NGen[0]] = np.array(Gen[iE].px)
    GenPy[0:NGen[0]] = np.array(Gen[iE].py)
    GenPz[0:NGen[0]] = np.array(Gen[iE].pz)
    GenE[0:NGen[0]] = np.array(Gen[iE].E)
    GenM[0:NGen[0]] = np.array(Gen[iE].mass)
    GenID[0:NGen[0]] = np.array(Gen[iE].id)
    GenStatus[0:NGen[0]] = np.array(Gen[iE].status)

    # Reco particles
    RecoPx[:]     = 0
    RecoPy[:]     = 0
    RecoPz[:]     = 0
    RecoE[:]      = 0
    RecoCharge[:] = 0
    RecoID[:]     = 0
    
    NReco[0] = len(Reco[iE].px)
    RecoPx[0:NReco[0]] = np.array(Reco[iE].px)
    RecoPy[0:NReco[0]] = np.array(Reco[iE].py)
    RecoPz[0:NReco[0]] = np.array(Reco[iE].pz)
    RecoE[0:NReco[0]] = np.array(Reco[iE].E)
    RecoCharge[0:NReco[0]] = np.array(Reco[iE].charge)
    RecoID[0:NReco[0]] = np.array(Reco[iE].id)

    # Reco particle IDs (only charged particles)
    RecoPIDElectron[:] = -999
    RecoPIDProton[:]   = -999
    RecoPIDPion[:]     = -999
    RecoPIDKaon[:]     = -999
    RecoPIDHeavy[:]    = -999
    RecoPIDQProton[:]  = -999
    RecoPIDQKaon[:]    = -999

    RecoPIDElectron[RecoCharge!=0] = np.array(PID[iE].ElectronTag)
    RecoPIDProton[RecoCharge!=0]   = np.array(PID[iE].ProtonTag)
    RecoPIDPion[RecoCharge!=0]     = np.array(PID[iE].PionTag)
    RecoPIDKaon[RecoCharge!=0]     = np.array(PID[iE].KaonTag)
    RecoPIDHeavy[RecoCharge!=0]    = np.array(PID[iE].HeavyTag)
    RecoPIDQProton[RecoCharge!=0]  = np.array(PID[iE].QProtonTag)
    RecoPIDQKaon[RecoCharge!=0]    = np.array(PID[iE].QKaonTag)

    # Gen-reco match
    GenMatchIndex[:] = -999
    GenMatchAngle[:] = -999
    
    for iG in range(NGen[0]):
        BestIndex = -1
        BestDistance = -1
        for iR in range(NReco[0]):
            Distance = GetAngle(GenPx[iG], GenPy[iG], GenPz[iG], RecoPx[iR], RecoPy[iR], RecoPz[iR])

            if BestIndex < 0 or BestDistance > Distance:
                BestIndex = iR
                BestDistance = Distance

        GenMatchIndex[iG] = BestIndex
        GenMatchAngle[iG] = BestDistance

    outputtree.Fill()

outputtree.Write()
outputfile.Close()